In [8]:
import sys
sys.path.append("..")

In [9]:
import graphSAGE_v0.random_graph as random_graph

In [10]:
node_df, edge_df = random_graph.random_graph_lcd(1000,3000,500)

In [22]:
edge_df

,cust_id,opp_id,weight
465,0,735,1.0
2427,0,716,1.0
1594,0,533,1.0
1570,0,873,1.0
2874,0,72,1.0
...,...,...,...
3415,998,28,1.0
4241,998,199,1.0
5656,999,646,1.0
5068,999,745,1.0


In [37]:
import networkx as nx

In [15]:
def ensure_integrity(graph: nx.classes.graph.Graph):
        """Ensure walk traversal conditions."""
        edge_list = [(index, index) for index in range(graph.number_of_nodes())]
        graph.add_edges_from(edge_list)

        return graph

In [16]:
def check_indexing(graph: nx.classes.graph.Graph):
        """Checking the consecutive numeric indexing."""
        numeric_indices = [index for index in range(graph.number_of_nodes())]
        node_indices = sorted([node for node in graph.nodes()])

        assert numeric_indices == node_indices, "The node indexing is wrong."

In [32]:
def check_graph(graph: nx.classes.graph.Graph):
        """Check the Karate Club assumptions about the graph."""
        check_indexing(graph)
        graph = ensure_integrity(graph)

        return graph

In [19]:
graph = nx.Graph() 

In [21]:
graph.add_nodes_from(node_df['cust_id']) 

In [26]:
graph.add_weighted_edges_from([(cust,opp,weight) for cust,opp,weight in zip(edge_df['cust_id'],edge_df['opp_id'],edge_df['weight'])])

In [34]:
graph = check_graph(graph)

In [48]:
import numpy as np

In [53]:
import random
# import community
import numpy as np
import networkx as nx
# from typing import Dict
# from karateclub.estimator import Estimator


class BigClam():
    r"""An implementation of `"BigClam" <http://infolab.stanford.edu/~crucis/pubs/paper-nmfagm.pdf>`_
    from the WSDM '13 paper "Overlapping Community Detection at Scale: A Non-negative Matrix
    Factorization Approach". The procedure uses gradient ascent to create an embedding which is
    used for deciding the node-cluster affiliations.
    Args:
        dimensions (int): Number of embedding dimensions. Default 8.
        iterations (int): Number of training iterations. Default 50.
        learning_rate (float): Gradient ascent learning rate. Default is 0.005.
        seed (int): Random seed value. Default is 42.
    """

    def __init__(
        self,
        dimensions: int = 8,
        iterations: int = 50,
        learning_rate: int = 0.005,
        seed: int = 42,
    ):
        self.dimensions = dimensions
        self.iterations = iterations
        self.learning_rate = learning_rate
        self.seed = seed

    def _initialize_features(self, number_of_nodes):
        """
        Creating the community embedding and gradient sum.
        Arg types:
            * **number_of_nodes** *(int)* - The number of nodes in the graph.
        """
        self._embedding = np.random.uniform(0, 1, (number_of_nodes, self.dimensions))
        self._global_features = np.sum(self._embedding, axis=0)

    def _calculate_gradient(self, node_feature, neb_features):
        """
        Calculating the feature gradient.
        Arg types:
            * **node_feature** *(Numpy array)* - The node representation.
            * **neb_features** *(Numpy array)* - The representation of node neighbours.
        """
        raw_scores = node_feature.dot(neb_features.T)
        raw_scores = np.clip(raw_scores, -15, 15)
        scores = np.exp(-raw_scores) / (1 - np.exp(-raw_scores))
        scores = scores.reshape(-1, 1)
        neb_grad = np.sum(scores * neb_features, axis=0)
        without_grad = (
            self._global_features - node_feature - np.sum(neb_features, axis=0)
        )
        grad = neb_grad - without_grad
        return grad

    def _do_updates(self, node, gradient, node_feature):
        """
        Updating the embedding and the feature sum.
        Arg types:
            * **node** *(int)* - The node identifier.
            * **gradient** *(Numpy array)* - The gradient of the node representation.
            * **node_feature** *(Numpy array)* - The node representation.
        """
        self._embedding[node] = self._embedding[node] + self.learning_rate * gradient
        self._embedding[node] = np.clip(self._embedding[node], 0.00001, 10)
        self._global_features = (
            self._global_features - node_feature + self._embedding[node]
        )

    def get_memberships(self):
        r"""Getting the cluster membership of nodes.
        Return types:
            * **memberships** *(dict)* - Node cluster memberships.
        """
        indices = np.argmax(self._embedding, axis=1)
        memberships = {i: membership for i, membership in enumerate(indices)}
        return memberships

    def get_embedding(self) -> np.array:
        r"""Getting the node embedding.
        Return types:
            * **embedding** *(Numpy array)* - The embedding of nodes.
        """
        embedding = self._embedding
        return embedding

    def fit(self, graph: nx.classes.graph.Graph):
        """
        Fitting a BigClam clustering model.
        Arg types:
            * **graph** *(NetworkX graph)* - The graph to be clustered.
        """
#         self._set_seed()
        graph = check_graph(graph)
        number_of_nodes = graph.number_of_nodes()
        self._initialize_features(number_of_nodes)
        nodes = [node for node in graph.nodes()]
        for i in range(self.iterations):
            random.shuffle(nodes)
            for node in nodes:
                nebs = [neb for neb in graph.neighbors(node)]
                neb_features = self._embedding[nebs, :]
                node_feature = self._embedding[node, :]
                gradient = self._calculate_gradient(node_feature, neb_features)
                self._do_updates(node, gradient, node_feature)

In [54]:
model = BigClam()

In [55]:
model.fit(graph)

In [66]:
from data import CoraData

In [67]:
data = CoraData(data_root="/Users/shuaihengxiao/Desktop/graphSAGE_v0/data/cora").data

Using Cached file: /Users/shuaihengxiao/Desktop/graphSAGE_v0/data/cora/ch7_cached.pkl


In [75]:
edge_lst = []
for key,value in data.adjacency_dict.items():
    for v in value:
        edge_lst.append((key,v))

In [87]:
graph = nx.Graph() 

In [88]:
graph.add_edges_from(edge_lst)

In [40]:
g = nx.newman_watts_strogatz_graph(1000,700, 0.01)

In [41]:
import bigclam

In [42]:
model = bigclam.BigClam(dimensions = 16, iterations = 100)

In [43]:
model.fit(g)

In [44]:
set(model.get_memberships().values())

{0}

In [45]:
model.get_embedding()

array([[10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.],
       ...,
       [10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.]])